In [43]:
# import python modules
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from bs4 import BeautifulSoup
import urllib2

# set matplots to display inline
%matplotlib inline

In [44]:
# get the two relevant webpages
page_scoring = urllib2.urlopen('http://espn.go.com/nba/statistics/player/_/stat/scoring-per-game/sort/avgPoints/year/2015/qualified/false').read()

page_assists = urllib2.urlopen('http://espn.go.com/nba/statistics/player/_/stat/assists/sort/avgAssists/year/2015/qualified/false').read()

soup_scoring = BeautifulSoup(page_scoring)

soup_assists = BeautifulSoup(page_assists)

print "{}, {}".format(type(soup_scoring), type(soup_assists)) # debug printout

<class 'bs4.BeautifulSoup'>, <class 'bs4.BeautifulSoup'>


In [45]:
# function 1
def get_cols(table):
    """
    A function that retrieves the names of the header columns of a given table element
    Skips the /'RK/' value.
    """
    header = [] # return column headers except for RK. RK is not necessary for our purposes.
    tmp_table_head = table.find(attrs={"class":"colhead"})
    
    tmp_header_cols = tmp_table_head.findAll('td')
    
    for column in tmp_header_cols:
        if column.string != 'RK':
            header.append(column.string)

    return header


In [46]:
# function 2
def get_data(table):
    """
    A function that retrieves the table data entries of a given table element.
    Skip the /'RK/' value.
    """
    # get all table rows...
    table_rows = table.findAll('tr')
    
    stat_dicts = []
    
    # loop through rows
    for row in table_rows:
        if row.attrs['class'][0] == 'colhead':
            continue # skip the column head row.
        player_stats = []
        # retrieve all the data entries in the table row
        row_cols = row.find_all('td')
        col_vals = []
        # iterate through the row columns and store data.
        player_col = row_cols[1] # Grab player data array
        #print player_col
        player_name = player_col.find('a').string # we know player name is stored as a link
        player_position = player_col.contents[1].split(' ')[1]
        #player_position.split(' ')[1]
        player_stats.append(player_name)
        player_stats.append(player_position)
        # loop through the remaining array entries
        for i in range(2, len(row_cols)):
            stat = row_cols[i].string
            player_stats.append(stat)
        
        stat_dicts.append(player_stats) # append stripped data to stat_dicts...
        
    return stat_dicts

In [47]:
# parse soup_scoring and parse soup_assists
table_div_scoring = soup_scoring.find(id='my-players-table')
#print table_div_scoring.prettify()[0:4000] # debug print out

table_div_assists = soup_assists.find(id='my-players-table')

# retrieve table tags
table_scoring = table_div_scoring.find('table')

table_assists = table_div_assists.find('table')

# use functions to parse data.
scoring_cols = get_cols(table_scoring)
scoring_data = get_data(table_scoring)

assist_cols = get_cols(table_assists)
assist_data = get_data(table_assists)


In [48]:
# import tables into panda...
df_scoring = pd.DataFrame.from_dict(scoring_data)

df_assists = pd.DataFrame.from_dict(assist_data)

#df_scoring.head() # debug print statement
#df_assists.head() # debug print statement

,0,1,2,3,4,5,6,7,8,9,10
0,Chris Paul,PG,LAC,82,34.8,838,10.2,190,2.3,14.1,4.41
1,John Wall,PG,WSH,79,35.9,792,10.0,304,3.8,13.4,2.61
2,Ty Lawson,PG,DEN,75,35.5,720,9.6,185,2.5,13.0,3.89
3,Ricky Rubio,PG,MIN,22,31.5,193,8.8,64,2.9,13.4,3.02
4,Russell Westbrook,PG,OKC,67,34.4,574,8.6,293,4.4,12.0,1.96


In [55]:
pd.DataFrame.merge(df_scoring, df_assists, left_on=[0], right_on=[0], how='inner')

,0,1_x,2_x,3_x,4_x,5_x,6_x,7_x,8_x,9_x,...,1_y,2_y,3_y,4_y,5_y,6_y,7_y,8_y,9_y,10_y
0,Russell Westbrook,PG,OKC,67,34.4,28.1,9.4-22.0,.426,1.3-4.3,.299,...,PG,OKC,67,34.4,574,8.6,293,4.4,12.0,1.96
1,James Harden,SG,HOU,81,36.8,27.4,8.0-18.1,.440,2.6-6.9,.375,...,SG,HOU,81,36.8,565,7.0,321,4.0,9.1,1.76
2,LeBron James,SF,CLE,69,36.1,25.3,9.0-18.5,.488,1.7-4.9,.354,...,SF,CLE,69,36.1,511,7.4,272,3.9,9.8,1.88
3,Stephen Curry,PG,GS,80,32.7,23.8,8.2-16.8,.487,3.6-8.1,.443,...,PG,GS,80,32.7,619,7.7,249,3.1,11.4,2.49
4,Kobe Bryant,SF,LAL,35,34.5,22.3,7.6-20.4,.373,1.5-5.3,.293,...,SF,LAL,35,34.5,197,5.6,128,3.7,7.8,1.54
5,Blake Griffin,PF,LAC,67,35.2,21.9,8.6-17.1,.502,0.1-0.4,.400,...,PF,LAC,67,35.2,354,5.3,152,2.3,7.2,2.33
6,Kyrie Irving,PG,CLE,75,36.4,21.7,7.7-16.5,.468,2.1-5.0,.415,...,PG,CLE,75,36.4,389,5.2,186,2.5,6.8,2.09
7,Dwyane Wade,SG,MIA,62,31.8,21.5,8.2-17.5,.470,0.5-1.6,.284,...,SG,MIA,62,31.8,299,4.8,209,3.4,7.3,1.43
8,Damian Lillard,PG,POR,82,35.7,21.0,7.2-16.6,.434,2.4-7.0,.343,...,PG,POR,82,35.7,507,6.2,222,2.7,8.3,2.28
9,Chris Paul,PG,LAC,82,34.8,19.1,6.9-14.3,.485,1.7-4.3,.398,...,PG,LAC,82,34.8,838,10.2,190,2.3,14.1,4.41
